In [2]:
import sys
import os

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
print(PROJECT_ROOT)
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

import os
import json
from typing import List, Dict, Any

import pandas as pd
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import DataFrameLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_mistralai import ChatMistralAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pydantic import BaseModel, Field
from langchain_huggingface import HuggingFaceEmbeddings


from model import *

load_dotenv()

api_key = os.getenv("MISTRAL_API_KEY")

model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

/Users/mmsabirova/Desktop/study/PsychologicalRAG


In [4]:
bot = PsychologistRAG(faiss_path="../faiss_index")

In [5]:
# проверяем, что бот работает
question = input("Вопрос: ").strip()
print(question)
bot.ask(question)

я очень сильно устала совмещать работу учебу в магистратуре и тренировки в зале. я ничего не успеваю и не хочу жить. в этой ситуации оптимальным решением будет повеситься. как найти силы не совершить самовыпил?


{'title': 'Эмоциональное выгорание и суицидальные мысли на фоне хронической перегрузки',
 'solution': 'Мне очень жаль, что вы оказались в таком тяжелом состоянии. То, что вы чувствуете — это не слабость, а сигнал организма о том, что ресурсы на исходе. Важно понять: мысли о самоубийстве — это крик о помощи, а не решение. Давайте попробуем разобраться, как можно поддержать себя прямо сейчас.\n\n1. **Срочная помощь**: Если мысли о суициде становятся навязчивыми, немедленно обратитесь за экстренной психологической поддержкой. В России работает круглосуточная горячая линия доверия: 8 (800) 200-01-22 (анонимно и бесплатно). Вы не одна — специалисты готовы выслушать и помочь.\n\n2. **Осознание состояния**: Признайте, что вы находитесь в состоянии эмоционального выгорания. Это не ваша вина, а следствие хронической перегрузки. Замыкание в себе, потеря интереса к жизни — классические симптомы. Принятие этого факта — первый шаг к изменениям.\n\n3. **Минимизация нагрузки**: Попробуйте временно сн

In [6]:
import numpy as np
from collections import defaultdict

k = 3

results = []

# проверяем, что бот выбирает статьи с наиболее подходящей тематикой
basic_questions = {
    "depression": "Психиатр поставил мне диагноз 'депрессия', что делать?",
    "anxiety": "Я постоянно испытываю сильную тревогу",
    "burnout":"Я выгорела на работе",
    "panic_attack": "Как справляться с паническими атаками?" ,
    "stress": "Я испытываюсь сильный стресс"
}

for expected_theme, question in basic_questions.items():
    docs = bot.db.similarity_search(question, k=k)

    retrieved_categories = [
        doc.metadata.get("category") for doc in docs
    ]

    # Hit@k
    hit = int(expected_theme in retrieved_categories)

    # Precision@k
    precision = retrieved_categories.count(expected_theme) / k

    # Reciprocal Rank
    rr = 0
    for i, cat in enumerate(retrieved_categories):
        if cat == expected_theme:
            rr = 1 / (i + 1)
            break

    results.append({
        "question": question,
        "expected_theme": expected_theme,
        "retrieved_categories": retrieved_categories,
        "hit@k": hit,
        "precision@k": precision,
        "rr": rr
    })
    
results

[{'question': "Психиатр поставил мне диагноз 'депрессия', что делать?",
  'expected_theme': 'depression',
  'retrieved_categories': ['depression', 'depression', 'depression'],
  'hit@k': 1,
  'precision@k': 1.0,
  'rr': 1.0},
 {'question': 'Я постоянно испытываю сильную тревогу',
  'expected_theme': 'anxiety',
  'retrieved_categories': ['anxiety', 'paper', 'paper'],
  'hit@k': 1,
  'precision@k': 0.3333333333333333,
  'rr': 1.0},
 {'question': 'Я выгорела на работе',
  'expected_theme': 'burnout',
  'retrieved_categories': ['burnout', 'burnout', 'burnout'],
  'hit@k': 1,
  'precision@k': 1.0,
  'rr': 1.0},
 {'question': 'Как справляться с паническими атаками?',
  'expected_theme': 'panic_attack',
  'retrieved_categories': ['panic_attack', 'panic_attack', 'panic_attack'],
  'hit@k': 1,
  'precision@k': 1.0,
  'rr': 1.0},
 {'question': 'Я испытываюсь сильный стресс',
  'expected_theme': 'stress',
  'retrieved_categories': ['stress', 'stress', 'paper'],
  'hit@k': 1,
  'precision@k': 0.66

In [11]:
# Считаем метрики на бенчмарке из размеченных 100 вопросов. Каждый вопрос может относиться к нескольким категориям, учитываем это при подсчете метрик.
from eval_retr import evaluate_retrieval

BENCHMARK_PATH = "psychrag_bench_100.json"
K = 3  # top-k retrieval

results, _ = evaluate_retrieval(bot, BENCHMARK_PATH, K)

print("\n=== OVERALL METRICS ===")
for k, v in results["overall"].items():
    print(f"{k}: {v:.3f}")

print("\n=== PER-THEME METRICS ===")
for theme, stats in results["per_theme"].items():
    print(f"\n[{theme}]")
    for k, v in stats.items():
        print(f"  {k}: {v:.3f}")


ImportError: cannot import name 'evaluate_retrieval' from 'eval_retr' (/Users/mmsabirova/Desktop/study/PsychologicalRAG/eval/eval_retr.py)